In [1]:
# data_loader.py
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def get_wasserball_ergebnisse():
    URL = "https://dsvdaten.dsv.de/Modules/WB/League.aspx?Season=2024&LeagueID=77&Group=&LeagueKind=L&StateID=17"
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")

    tables = soup.find_all("table")
    results = []

    for table in tables:
        for row in table.find_all("tr")[1:]:
            cols = row.find_all("td")
            if len(cols) >= 7:
                spielnummer = cols[0].text.strip()
                datum_uhrzeit = cols[1].text.strip()
                heim = cols[2].text.strip()
                gast = cols[3].text.strip()
                ort = cols[4].text.strip()

                ergebnis_tag = cols[5].find("a")
                ergebnis = ergebnis_tag.text.strip() if ergebnis_tag else ""
                protokoll_link = ergebnis_tag.get("href", "") if ergebnis_tag else ""

                viertel = cols[6].text.strip()

                match = re.findall(r'\d+', ergebnis)
                if len(match) >= 2:
                    heim_tore, gast_tore = int(match[0]), int(match[1])
                else:
                    heim_tore, gast_tore = None, None

                results.append({
                    "Spielnummer": spielnummer,
                    "Datum & Uhrzeit": datum_uhrzeit,
                    "Heim": heim,
                    "Gast": gast,
                    "Ort": ort,
                    "Ergebnis": ergebnis,
                    "Heim_Tore": heim_tore,
                    "Gast_Tore": gast_tore,
                    "Protokoll-Link": protokoll_link,
                    "Viertelergebnisse": viertel
                })

    return pd.DataFrame(results)

In [16]:
def cleanse_data(df):
    #datum extrahieren
    df[["Datum", "Uhrzeit"]] = df["Datum & Uhrzeit"].str.split(" ", n=1, expand=True)
    df["Datum"] = pd.to_datetime(df["Datum"], format="%d.%m.%Y", dayfirst=True, errors="coerce")

    #Tordifferenz berechnen
    df["Heim_Tore"] = df["Ergebnis"].str.extract(r"(\d+)", expand=False).astype(float)
    df["Gast_Tore"] = df["Ergebnis"].str.extract(r":(\d+)", expand=False).astype(float)
    df["Tordifferenz"] = df["Heim_Tore"] - df["Gast_Tore"]

    #Spielstatus bestimmen
    df["Status"] = df["Ergebnis"].apply(lambda x: "Gespielt" if ":" in str(x) else "Offen")

    viertel = df["Viertelergebnisse"].str.split(",", expand=True)
    for i, name in enumerate(["Q1", "Q2", "Q3", "Q4"]):
        df[f"{name}_Heim"] = viertel[i].str.extract(r"(\d+)", expand=False).astype(float)
        df[f"{name}_Gast"] = viertel[i].str.extract(r":(\d+)", expand=False).astype(float)

    return df

In [3]:
def berechne_teamstatistiken(df):
    teams = pd.unique(df[["Heim", "Gast"]].values.ravel())
    stats_liste = []

    for team in teams:
        spiele = df[(df["Heim"] == team) | (df["Gast"] == team)].copy()

        spiele["Eigene_Tore"] = spiele.apply(
            lambda row: row["Heim_Tore"] if row["Heim"] == team else row["Gast_Tore"], axis=1)
        spiele["Gegentore"] = spiele.apply(
            lambda row: row["Gast_Tore"] if row["Heim"] == team else row["Heim_Tore"], axis=1)

        spiele["Ergebnis_Typ"] = spiele.apply(
            lambda row: "Sieg" if row["Eigene_Tore"] > row["Gegentore"]
            else ("Unentschieden" if row["Eigene_Tore"] == row["Gegentore"] else "Niederlage"),
            axis=1)

        # Viertel: Eigene und Gegentore je Viertel aggregieren
        eigene_viertel = {"Q1": 0, "Q2": 0, "Q3": 0, "Q4": 0}
        gegner_viertel = {"Q1": 0, "Q2": 0, "Q3": 0, "Q4": 0}

        for i in range(1, 5):
            q = f"Q{i}"
            eigene_viertel[q] = spiele.apply(
                lambda row: row[f"{q}_Heim"] if row["Heim"] == team else row[f"{q}_Gast"], axis=1).sum()

            gegner_viertel[q] = spiele.apply(
                lambda row: row[f"{q}_Gast"] if row["Heim"] == team else row[f"{q}_Heim"], axis=1).sum()

        stats = {
            "Team": team,
            "Spiele_gesamt": len(spiele),
            "Gespielt": (spiele["Status"] == "Gespielt").sum(),
            "Offen": (spiele["Status"] == "Offen").sum(),
            "Tore_ges": spiele["Eigene_Tore"].sum(),
            "Gegentore_ges": spiele["Gegentore"].sum(),
            "Ø Tore": round(spiele["Eigene_Tore"].mean(), 2),
            "Ø Gegentore": round(spiele["Gegentore"].mean(), 2),
            "Tordifferenz": spiele["Eigene_Tore"].sum() - spiele["Gegentore"].sum(),
            "Siege": (spiele["Ergebnis_Typ"] == "Sieg").sum(),
            "Unentschieden": (spiele["Ergebnis_Typ"] == "Unentschieden").sum(),
            "Niederlagen": (spiele["Ergebnis_Typ"] == "Niederlage").sum()
        }

        # Vierteldaten hinzufügen
        for i in range(1, 5):
            stats[f"Q{i}_Tore"] = eigene_viertel[f"Q{i}"]
            stats[f"Q{i}_Gegentore"] = gegner_viertel[f"Q{i}"]

        stats_liste.append(stats)

    return pd.DataFrame(stats_liste)

In [4]:
df = get_wasserball_ergebnisse()

In [18]:
df = cleanse_data(df)

In [22]:
df_stat = berechne_teamstatistiken(df)

In [26]:
df_stat.head(10)

,Team,Spiele_gesamt,Gespielt,Offen,Tore_ges,Gegentore_ges,Ø Tore,Ø Gegentore,Tordifferenz,Siege,Unentschieden,Niederlagen,Q1_Tore,Q1_Gegentore,Q2_Tore,Q2_Gegentore,Q3_Tore,Q3_Gegentore,Q4_Tore,Q4_Gegentore
0,Aachener SV 06,16,9,7,59.0,129.0,6.56,14.33,-70.0,1,0,15,15.0,34.0,8.0,29.0,14.0,35.0,12.0,31.0
1,SV Krefeld 1972 II,16,8,8,113.0,93.0,14.12,11.62,20.0,6,0,10,31.0,23.0,26.0,22.0,30.0,17.0,26.0,21.0
2,ASC Duisburg II,16,8,8,118.0,112.0,14.75,14.00,6.0,3,0,13,25.0,23.0,31.0,20.0,34.0,27.0,28.0,29.0
3,Duisburger SV 1898 II,16,6,10,72.0,57.0,12.00,9.50,15.0,4,0,12,20.0,14.0,16.0,16.0,15.0,11.0,21.0,16.0
4,SG Wasserball Essen,16,9,7,88.0,127.0,9.78,14.11,-39.0,2,0,14,16.0,28.0,25.0,35.0,17.0,36.0,20.0,28.0
5,SGW SC Solingen/Wfr. Wuppertal,16,8,8,79.0,103.0,9.88,12.88,-24.0,4,0,12,18.0,21.0,17.0,23.0,21.0,31.0,23.0,28.0
6,SGW Rhenania/BW Poseidon Köln,16,9,7,144.0,90.0,16.00,10.00,54.0,7,0,9,29.0,26.0,31.0,20.0,43.0,20.0,38.0,24.0
7,SV Lünen 08,16,9,7,108.0,128.0,12.00,14.22,-20.0,2,0,14,30.0,30.0,25.0,30.0,22.0,34.0,31.0,34.0
8,SV Blau-Weiß Bochum,16,8,8,138.0,80.0,17.25,10.00,58.0,8,0,8,36.0,21.0,30.0,14.0,37.0,22.0,35.0,23.0
9,8/16,4,0,4,0.0,0.0,NaN,NaN,0.0,0,0,4,448.0,385.0,0.0,0.0,0.0,0.0,0.0,0.0
